In [1]:
import re
import pandas as pd
import numpy as np
from starfit import Single, Multi
from pathlib import Path
import socket
import matplotlib.pyplot as plt
from starfit.utils import find_data
from starfit.autils.stardb import StarDB
from starfit.autils.abusets import SolAbu
from astropy import constants as const
from astropy import units
coeff = (0.6 * (const.G * const.M_sun) / (12e3 * units.m * (const.c)**2)).value

import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

hostname = socket.gethostname()
if hostname == 'jerome-linux':
    datadir = Path('/home/jerome/Documents/GitHub')
elif hostname == 'sage2020':
    datadir = Path('/home/jiangrz/hdd23')
datapath = datadir/Path('Data/rproc/heavy_element_enhanced_stars_write_err.csv')
stardir = datadir/Path('Data/rproc/star')
lightdir = datadir/Path('Data/Raw/MPs/1_Li/Stars')
resdir = Path('result/')
df = pd.read_csv(datapath)
database = [
    'rproc.wu.star.el.y.stardb.xz', 
    'rproc.just15.star.el.y.stardb.xz', 
    'nuc.lc18.star.el.y.stardb.xz', 
    'znuc2012.S4.star.el.y.stardb.gz', 
]
z_exclude_lc18 = np.append(np.arange(43, 54), np.arange(61, 80)).tolist()
z_exclude_s4 = [21, 24, 29, 30]

In [5]:
def run(dbnames, z_exclude, resfile, constraints, z_min, z_max):
    columns = []
    properties = ['name', 'FeH', 'N', 'fitness']
    freeparams_num = 0
    columns.extend([('property', prop) for prop in properties])
    for _didx, _dname in enumerate(dbnames):
        dbpath = find_data('db', _dname)
        _db = StarDB(dbpath, silent=True)
        freeparams_num += np.sum(1-_db.fieldflags)
        fieldnames = _db.fieldnames
        columns.extend([('%d'%_didx, _) for _ in np.append(fieldnames, 'offset')])
        df_result = pd.DataFrame(columns=columns)
        df_result.columns = pd.MultiIndex.from_tuples(df_result.columns, names=['db_index','fieldname'])

    for idx_row, obj_row in df.iterrows():
        starname = obj_row.objname.split('.')[0]
        starFeH = obj_row.FeH
        starpath = stardir/('%s.dat'%(starname))
        comb = Single(
            starpath, db=dbnames, 
            constraints=constraints, 
            z_min=z_min, z_max=z_max, z_exclude=z_exclude, 
            silent=True
        )

        starN = comb.star.element_abundances[~comb.exclude_index].shape[0]
        dof = starN - freeparams_num - len(dbnames)

        starfitn = comb.sorted_fitness[0] / dof
        starindices = comb.sorted_stars[0]['index']
        staroffsets = comb.sorted_stars[0]['offset']
        db_idx = comb.db_idx[starindices]
        db_off = comb.db_off
        starindices -= db_off
        starprop = [starname, starFeH, starN, starfitn]
        columndata = starprop
        for _db_idx, _staridx, _staroffset in zip(db_idx, starindices, staroffsets):
            starfield = list(comb.db[_db_idx].fielddata[_staridx])
            stardata = starfield + [_staroffset]
            columndata += stardata
        df_result.loc[len(df_result)] = columndata
        if starfitn < 3:
            comb.plot(xlim=(0, 84), ylim=(-5, 2))
    df_result.to_csv(resdir/resfile, index=False)

In [3]:
dbnames = ['nuc.lc18.star.el.y.stardb.xz']
z_exclude = [1, 2, 3, 6]+z_exclude_lc18
constraints = '0:metallicity<0.1'
# run(dbnames, z_exclude, 'res_lc18_under50.csv', constraints, 3, 50)

NameError: name 'z_exclude_lc18' is not defined

In [2]:
dbnames = ['rproc.just15.star.el.y.stardb.xz']
z_exclude = None
constraints = None
# run(dbnames, z_exclude, 'res_just_over50.csv', constraints, 51, 85)

In [1]:
# dbnames = ['rproc.wu.star.el.y.stardb.xz']
# z_exclude = None
# constraints = None
# run(dbnames, z_exclude, 'res_wu_over50.csv', constraints,  50, 85)

In [35]:
df_dict = {
    'lc18': pd.read_csv(resdir/'res_lc18_under50.csv', header=[0, 1]), 
    'just': pd.read_csv(resdir/'res_just_over50.csv', header=[0, 1]),
    'wu': pd.read_csv(resdir/'res_wu_over50.csv', header=[0, 1])
}
fitn_arr = np.ndarray((len(df_dict), len(df)))
for idx_arr, key_model in enumerate(['lc18', 'just', 'wu']):
    fitn_arr[idx_arr, :] = df_dict[key_model].loc[:, ('property', 'fitness')].values

In [32]:
threshold = 1
mask = np.any((fitn_arr>threshold, fitn_arr<0), axis=0)
fitn_maarr = np.ma.array(fitn_arr, mask=mask)
flag_just = np.any(~fitn_maarr.mask[1:2, :], axis=0)
flag_wu = np.any(~fitn_maarr.mask[2:, :], axis=0)
flag_rotsn = np.any(~fitn_maarr.mask[:1, :], axis=0)
flag_onlyjust = flag_just & ~ flag_wu & ~ flag_rotsn
flag_onlywu = ~ flag_just & flag_wu & ~ flag_rotsn
flag_justwu = flag_just & flag_wu & ~ flag_rotsn
flag_onlyrotsn = ~ flag_just & ~ flag_wu & flag_rotsn
flag_all = (flag_just | flag_wu) & flag_rotsn
objnames = df_dict['lc18'].loc[:, ('property', 'name')].values
print('NSM_just  :%3d\n'%np.sum(flag_onlyjust), objnames[flag_onlyjust])
print('NSM_wu    :%3d\n'%np.sum(flag_onlywu), objnames[flag_onlywu])
print('NSM_justwu:%3d\n'%np.sum(flag_justwu), objnames[flag_justwu])
print('rCCSN     :%3d\n'%np.sum(flag_onlyrotsn), objnames[flag_onlyrotsn])
print('NSM/rCCSN :%3d\n'%np.sum(flag_all), objnames[flag_all])

NSM_just  :  0
 []
NSM_wu    : 22
 ['J0003+1556' 'J0041+3633' 'J0127+1100' 'J0643+5934' 'J1109+0754'
 'J1208+3954' 'J1218+2838' 'J1226+2323' 'J1236+3424' 'J1341+3255'
 'J1414+1457' 'J1423+0322' 'J1459+0444' 'J1523+0714' 'J1612-0207'
 'J1634+0206' 'J1640+1550' 'J1641+0334' 'J1730+4143' 'J1833+1309'
 'J2159+0308' 'J2216+0246']
NSM_justwu:  1
 ['J0949+2707']
rCCSN     :  0
 []
NSM/rCCSN :  0
 []


In [36]:
threshold = 3
mask = np.any((fitn_arr>threshold, fitn_arr<0), axis=0)
fitn_maarr = np.ma.array(fitn_arr, mask=mask)
flag_just = np.any(~fitn_maarr.mask[1:2, :], axis=0)
flag_wu = np.any(~fitn_maarr.mask[2:, :], axis=0)
flag_rotsn = np.any(~fitn_maarr.mask[:1, :], axis=0)
flag_onlyjust = flag_just & ~ flag_wu & ~ flag_rotsn
flag_onlywu = ~ flag_just & flag_wu & ~ flag_rotsn
flag_justwu = flag_just & flag_wu & ~ flag_rotsn
flag_onlyrotsn = ~ flag_just & ~ flag_wu & flag_rotsn
flag_all = (flag_just | flag_wu) & flag_rotsn
objnames = df_dict['lc18'].loc[:, ('property', 'name')].values
print('NSM_just  :%3d\n'%np.sum(flag_onlyjust), objnames[flag_onlyjust])
print('NSM_wu    :%3d\n'%np.sum(flag_onlywu), objnames[flag_onlywu])
print('NSM_justwu:%3d\n'%np.sum(flag_justwu), objnames[flag_justwu])
print('rCCSN     :%3d\n'%np.sum(flag_onlyrotsn), objnames[flag_onlyrotsn])
print('NSM/rCCSN :%3d\n'%np.sum(flag_all), objnames[flag_all])

NSM_just  :  2
 ['J1020+4046' 'J2333+0505']
NSM_wu    : 52
 ['J0003+1556' 'J0040+2729' 'J0041+3633' 'J0055+1857' 'J0124+0458'
 'J0127+1100' 'J0302+1356' 'J0405+2141' 'J0446+2124' 'J0554+5235'
 'J0643+5934' 'J0804+5740' 'J0934-0108' 'J0948+0000' 'J1044+3713'
 'J1059+0208' 'J1101+2031' 'J1122+1223' 'J1145+0359' 'J1207+2244'
 'J1208+3954' 'J1218+2838' 'J1221+0907' 'J1225-0452' 'J1226+2323'
 'J1236+3424' 'J1250-0307' 'J1345+0513' 'J1352+2646' 'J1400+1215'
 'J1413+1727' 'J1414+1457' 'J1414+1721' 'J1423+0322' 'J1459+0444'
 'J1501+3445' 'J1516+0442' 'J1518+2544' 'J1520+5500' 'J1523+0714'
 'J1604+1305' 'J1612-0207' 'J1634+0206' 'J1640+1550' 'J1657+3443'
 'J1717+2449' 'J1719+1358' 'J1730+4143' 'J1745+5410' 'J1833+1309'
 'J2216+0246' 'J2347+2851']
NSM_justwu:  4
 ['J0949+2707' 'J1109+0754' 'J1158+0734' 'J2159+0308']
rCCSN     :  0
 []
NSM/rCCSN :  5
 ['J0711+3538' 'J1341+3255' 'J1641+0334' 'J1650+4313' 'J1702+1732']
